In [228]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import random

data = pd.read_csv("Sentiment_Analysis_data.csv")
data.fillna("", inplace=True)


In [229]:
print(data)

      Unnamed: 0         Customer Name                cust mail id  \
0              0          Anna Ramirez        nathan18@example.net   
1              1        Ricky Caldwell  millercharlene@example.net   
2              2          Richard Ford      jonesjason@example.com   
3              3           Maria Lopez  amandacampbell@example.net   
4              4  Christopher Anderson         karen57@example.org   
...          ...                   ...                         ...   
2295        2295          Taylor Hogan          neil21@example.net   
2296        2296          Richard York         cbutler@example.com   
2297        2297           Stacy Black    sarahbennett@example.com   
2298        2298         Marcus Barton       handerson@example.net   
2299        2299       Samuel Shepherd      yhernandez@example.net   

                                               Feedback     Date and time  \
0     beautiful accomodations stayed hotel santa mar...  2024-10-19 17:07   
1    

In [230]:
# Step 1: Generate User Data
import random
def generate_user_data(num_users=500, num_days=30):
    user_data = []

    # Define possible activities and their categories
    activities = {
        'amenities': ['pool', 'spa', 'gym', 'tennis_court', 'business_center'],
        'dining': ['main_restaurant', 'cafe', 'bar', 'room_service', 'buffet'],
        'activities': ['city_tour', 'beach_activity', 'cooking_class', 'yoga', 'golf']
    }

    # Generate random user interactions
    # start_date = datetime.now() - timedelta(days=num_days)

    for user_id in range(num_users):
        # User preferences (1-5 rating)
        for category in activities:
            for activity in activities[category]:
                # Not all users will have interactions with all activities
                
                    user_data.append({
                        'user_id': f'{user_id}',
                        'category': category,
                        'activity': activity,
                        'rating': random.randint(1, 5),
                    })

    return pd.DataFrame(user_data)

In [231]:
data = generate_user_data()
data

,user_id,category,activity,rating
0,0,amenities,pool,2
1,0,amenities,spa,1
2,0,amenities,gym,5
3,0,amenities,tennis_court,3
4,0,amenities,business_center,3
...,...,...,...,...
7495,499,activities,city_tour,5
7496,499,activities,beach_activity,2
7497,499,activities,cooking_class,4
7498,499,activities,yoga,2


In [232]:
data.to_csv("recommendation_data.csv")

In [233]:
data[data['user_id'] == "user_0"]['activity'].value_counts()

Series([], Name: count, dtype: int64)

In [234]:
# Step 2: Build User Profiles and Similarity Matrix
def build_user_profiles(data):
    
    # Create a user-activity matrix
    user_profiles = data.pivot_table(index='user_id', columns='activity', values='rating').fillna(0)

    # Compute similarity matrix using cosine similarity
    similarity_matrix = cosine_similarity(user_profiles)
    return similarity_matrix, user_profiles



In [235]:
print(data.head())


  user_id   category         activity  rating
0       0  amenities             pool       2
1       0  amenities              spa       1
2       0  amenities              gym       5
3       0  amenities     tennis_court       3
4       0  amenities  business_center       3


In [236]:
print(data['rating'].dtype)


int64


In [237]:
similarity_matrix, user_profiles = build_user_profiles(data)
print(similarity_matrix)


[[1.         0.86067219 0.85759927 ... 0.85895799 0.88923552 0.82629363]
 [0.86067219 1.         0.87989809 ... 0.93861684 0.89080208 0.89941829]
 [0.85759927 0.87989809 1.         ... 0.83437103 0.83681286 0.80976187]
 ...
 [0.85895799 0.93861684 0.83437103 ... 1.         0.84339432 0.857266  ]
 [0.88923552 0.89080208 0.83681286 ... 0.84339432 1.         0.84956754]
 [0.82629363 0.89941829 0.80976187 ... 0.857266   0.84956754 1.        ]]


In [238]:
user_profiles

activity,bar,beach_activity,buffet,business_center,cafe,city_tour,cooking_class,golf,gym,main_restaurant,pool,room_service,spa,tennis_court,yoga
user_id,,,,,,,,,,,,,,,
0,3.0,2.0,3.0,3.0,2.0,3.0,4.0,1.0,5.0,2.0,2.0,4.0,1.0,3.0,3.0
1,1.0,4.0,3.0,5.0,4.0,4.0,2.0,4.0,3.0,5.0,5.0,5.0,1.0,5.0,2.0
10,2.0,1.0,3.0,5.0,1.0,5.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,5.0,2.0
100,1.0,3.0,3.0,1.0,1.0,3.0,5.0,4.0,3.0,2.0,4.0,4.0,3.0,4.0,5.0
101,4.0,5.0,3.0,5.0,3.0,3.0,4.0,4.0,3.0,1.0,1.0,1.0,4.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,3.0,5.0,4.0,4.0,5.0,1.0,3.0,1.0,5.0,4.0,2.0,4.0,2.0,5.0
96,5.0,1.0,1.0,5.0,3.0,5.0,4.0,1.0,3.0,5.0,2.0,3.0,4.0,3.0,2.0
97,1.0,4.0,5.0,3.0,3.0,2.0,1.0,3.0,3.0,4.0,5.0,5.0,2.0,5.0,5.0


In [242]:
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Generate User Data
def generate_user_data(num_users=500, num_days=30):
    
    user_data = []

    # Define possible activities and their categories
    activities = {
        'amenities': ['pool', 'spa', 'gym', 'tennis_court', 'business_center'],
        'dining': ['main_restaurant', 'cafe', 'bar', 'room_service', 'buffet'],
        'activities': ['city_tour', 'beach_activity', 'cooking_class', 'yoga', 'golf']
    }

    # Generate random user interactions
    for user_id in range(num_users):
        for category in activities:
            for activity in activities[category]:
                # Randomly decide if the user interacted with the activity
                if random.random() < 0.3:  # 30% chance of interaction
                    user_data.append({
                        'user_id': f'{user_id}',
                        'category': category,
                        'activity': activity,
                        'rating': random.randint(1, 5),
                    })

    return pd.DataFrame(user_data)

# Step 2: Build User Profiles and Similarity Matrix
def build_user_profiles(data):
  
    # Create a user-activity matrix
    user_profiles = data.pivot_table(index='user_id', columns='activity', values='rating').fillna(0)

    # Compute similarity matrix using cosine similarity
    similarity_matrix = cosine_similarity(user_profiles)
    return similarity_matrix, user_profiles

# Step 3: Find Similar Users
def get_similar_users(data, user_id, n=5, similarity_matrix=None):
    
    if similarity_matrix is None:
        similarity_matrix, user_profiles = build_user_profiles(data)

    user_profiles = data.pivot_table(index='user_id', columns='activity', values='rating').fillna(0)
    user_idx = user_profiles.index.get_loc(user_id)
    user_similarities = similarity_matrix[user_idx]

    # Get indices of the top `n` similar users (excluding the user itself).
    similar_user_indices = user_similarities.argsort()[::-1][1:n + 1]
    similar_users = user_profiles.index[similar_user_indices]

    return list(similar_users)

# Step 4: Generate Recommendations
def get_recommendations(data, user_id, n=5, similarity_matrix=None):
    
    if similarity_matrix is None:
        similarity_matrix, user_profiles = build_user_profiles(data)

    similar_users = get_similar_users(data, user_id, n=5, similarity_matrix=similarity_matrix)

    # Filter activities rated by similar users
    similar_users_data = data[data['user_id'].isin(similar_users)]

    # Calculate average ratings for activities
    recommendations = similar_users_data.groupby('activity').agg({
        'rating': 'mean'
    }).sort_values('rating', ascending=False)

    # Filter out activities the user has already tried
    user_activities = set(data[data['user_id'] == user_id]['activity'])
    new_activities = recommendations[~recommendations.index.isin(user_activities)]

    return new_activities.head(n)

# Step 5: User Interaction
if __name__ == "__main__":
    # Generate synthetic user data
    data = generate_user_data(num_users=100, num_days=30)

    # User input
    user_id = input("Enter your user ID (e.g., '0'): ")

    # Generate recommendations
    similarity_matrix, _ = build_user_profiles(data)
    recommendations = get_recommendations(data, user_id, n=10, similarity_matrix=similarity_matrix)

    # Output
    if recommendations.empty:
        print(f"No new recommendations available for User {user_id}.")
    else:
        print(f"Recommendations for User {user_id}:")
        print(recommendations)


Recommendations for User 6:
                 rating
activity               
pool                5.0
beach_activity      4.0
buffet              4.0
city_tour           4.0
business_center     3.0
golf                3.0
yoga                1.0


In [244]:
from flask import Flask, render_template, request, jsonify
from recsys import generate_user_data, get_recommendations, build_user_profiles  # Import from your recsys file
import pandas as pd

app = Flask(__name__)

# Generate synthetic data and build similarity matrix
data = generate_user_data(num_users=100)
similarity_matrix, _ = build_user_profiles(data)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    user_id = request.form.get('user_id')

    # Ensure the user ID exists in the data
    if user_id not in data['user_id'].unique():
        return jsonify({"error": "User ID not found!"}), 404

    # Get recommendations
    recommendations = get_recommendations(data, user_id, n=5, similarity_matrix=similarity_matrix)

    if not recommendations:
        return jsonify({"message": f"No recommendations available for User {user_id}."})

    return jsonify({"recommendations": recommendations})

if __name__ == '__main__':
    app.run(debug=True)


Recommendations for User 3:
                 rating
activity               
beach_activity      5.0
bar                 1.0
business_center     1.0
city_tour           1.0
room_service        1.0
spa                 1.0
tennis_court        1.0
